In [16]:
import pandas as pd
import re
from rapidfuzz import fuzz

# --- 1. Define the cleaning function ---
def clean_school_name(name):
    if pd.isna(name):
        return "Unknown"
    
    # Fix HTML entities and encoding issues
    name = name.replace("&#039;", "'").replace("&apos;", "'").replace("&amp;", "and").replace("’", "'")
    
    # Lowercase for uniformity during grouping
    name = name.lower()
    
    # Remove numbers, parentheses, and all special characters (except space)
    name = re.sub(r"[0-9]", "", name)
    name = re.sub(r"\([^)]*\)", "", name)  # remove text inside parentheses
    name = re.sub(r"[^a-zA-Z\s]", "", name)  # remove all except letters and spaces
    
    # Normalize multiple spaces
    name = re.sub(r"\s+", " ", name).strip()
    
    return name


# --- 2. Fuzzy group similar names ---
def fuzzy_group_school_names(school_list, threshold=70):  # Lower threshold
    grouped = []
    seen = set()
    
    for i, school in enumerate(school_list):
        if school in seen:
            continue
        group = [school]
        seen.add(school)
        
        for other in school_list[i+1:]:
            if other not in seen and fuzz.ratio(school, other) >= threshold:
                group.append(other)
                seen.add(other)
        
        grouped.append(group)
    
    return grouped


# --- 3. Build canonical mapping ---
def build_canonical_school_map(grouped_list):
    mapping = {}
    for group in grouped_list:
        canonical = max(group, key=len)  # you can choose other heuristics too
        for name in group:
            mapping[name] = canonical
    return mapping

# --- 4. Load the CSV ---
df = pd.read_csv("C:/Thesis Refactor/APP/dataset/college.csv")

if "previous_school" not in df.columns:
    raise ValueError("Column 'previous_school' not found in the CSV.")

# --- 5. Clean and standardize ---
df["cleaned_school"] = df["previous_school"].astype(str).apply(clean_school_name)
unique_cleaned_schools = df["cleaned_school"].unique().tolist()

grouped = fuzzy_group_school_names(unique_cleaned_schools, threshold=90)
canonical_map = build_canonical_school_map(grouped)

df["standardized_school"] = df["cleaned_school"].map(canonical_map).str.upper()

# --- 6. Visualize the results ---
df[["previous_school", "cleaned_school", "standardized_school"]] \
    .drop_duplicates() \
    .sort_values("standardized_school") \
    .head(30)  # Adjust or remove head() to see more

# --- 7. Optional: Get top 10 standardized schools ---
top_schools = df["standardized_school"].value_counts().head(10)
print("Top 10 Schools by Student Count:")
print(top_schools)


Top 10 Schools by Student Count:
standardized_school
NAN                                         432
UNIVERSITY OF CEBU LAPU LAPU AND MANDAUE    247
UNIVERSITY OF CEBU PRI                      235
CEBU INSTITITUTE OF TECHNOLOGY UNIVERSIT    224
SOUTHWESTERN UNIVERSITY PHINMA SWU          185
SINDANGAN NATIONAL HIGH SCHOOL              153
ASIAN COLLEGE OF TECHNOLOGY MAIN            121
UNIVERSITY OF CEBU BANILAD                  115
ST ALPHONSUS CATHOLIC SCHOOL SACS           113
UNIVERSITY OF CEBU MAIN SHS CAMPUS          113
Name: count, dtype: int64


In [17]:
# View original vs cleaned vs standardized previous school names (no duplicates)
school_comparison = df[["previous_school", "cleaned_school", "standardized_school"]] \
    .drop_duplicates() \
    .sort_values("standardized_school") \
    .reset_index(drop=True)

# Show the full comparison
pd.set_option('display.max_rows', None)  # Optional: show all rows
school_comparison


,previous_school,cleaned_school,standardized_school
0,2023,,
1,20142015,,
2,2020,,
3,2022,,
4,20222023,,
5,20202021,,
6,20222024,,
7,20202022,,
8,2024,,
9,2021,,
